# Segmenting and Clustering Neighborhoods in Toronto

<h3>Introduction</h3>
<p>
This is an assignment for the Introduction to Artifical Intelligence course (SOFE 3720U). Within this we will be explore how to segement and cluster the neighborhoods in Toronto. 
</p>

<h3>Import Statements</h3>

In [3]:
from dotenv import load_dotenv
from dotenv import dotenv_values

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import geojson

import requests
from pandas import json_normalize

import folium

from bs4 import BeautifulSoup as bs

print('Libraries Imported')

Libraries Imported


<h3>Week 1 - Foursquare API</h3>
<p>
Within this section we will be using the Foursquare API to find latitude, longitude, and venues within the Toronto area.
</p>

<h4>
Setting up Foursquare API
</h4>

In [4]:
config = dotenv_values(".env")
url = "https://api.foursquare.com/v3/places/nearby"

headers = {"Accept": "application/json",
            "Authorization": config["API_KEY"]}

response = requests.request("GET", url, headers=headers)

def findNearbyVenues(location, categories, limit):
    url = "https://api.foursquare.com/v3/places/search?" + "categories=" + categories + "&near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    response = requests.request("GET", url, headers=headers)

    if(response.status_code == 200):
        return response.json()
    else:
        return False

<h4>
Creating dataframe using the function created to use Foursquare API
</h4>

In [9]:
latitude = 43.6532 
longitude = -79.3832
results = findNearbyVenues(location = ["Toronto", "ON"], categories="17000", limit="50")

venuesdf = json_normalize(results['results'], max_level=3)
venuesdf.drop(venuesdf.columns[[1,2,3,5,6,7,10,12,13,18]], axis=1, inplace=True)
venuesdf.head()

,fsq_id,name,geocodes.roof.latitude,geocodes.roof.longitude,location.country,location.locality,location.neighborhood,location.postcode,location.region,related_places.children,related_places.parent.fsq_id,related_places.parent.name,location.po_box
0,4adcfd7cf964a5203e6321e3,Fiesta Farms,43.668877,-79.420664,CA,Toronto,[Christie Pitts],M6G 3B6,ON,NaN,NaN,NaN,NaN
1,4ad4c062f964a52002f820e3,Yorkdale,NaN,NaN,CA,Toronto,[Downsview],M6A 2T9,ON,"[{'fsq_id': '51ab9740498e5d77772d110b', 'name'...",NaN,NaN,NaN
2,580a6d5fd67cd1299d4aa14f,Uniqlo ユニクロ,NaN,NaN,CA,Toronto,"[Lawrence Heights, Toronto, ON]",M6A 2T9,ON,NaN,4ad4c062f964a52002f820e3,Yorkdale,NaN
3,5a6f54bbf193c029b1ecd567,Loblaws,43.661965,-79.379811,CA,Toronto,NaN,M5B 1J2,ON,"[{'fsq_id': '4ed6319130f813e77a9f8a3b', 'name'...",NaN,NaN,NaN
4,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,43.653807,-79.380056,CA,Toronto,[Eaton Centre],M5B 2H1,ON,"[{'fsq_id': '531e0a48498e172fbe0c59da', 'name'...",NaN,NaN,PO Box 511


In [88]:
map_creation = folium.Map(location=[latitude, longitude], zoom_start=10)
map_creation

<h3>Week 2 - Prepare your data</h3>
<p>
Within this section we will be using the provide source to create a large dataframe which contains the neccessary information for the choosen correlations
</p>

In [20]:
"""url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'
temp = requests.get(url)
data = temp.text
soup = bs(data,'html.parser')
wiki = soup.find('table')
df = pd.read_html(str(wiki))[0]
df.head()"""

"url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'\ntemp = requests.get(url)\ndata = temp.text\nsoup = bs(data,'html.parser')\nwiki = soup.find('table')\ndf = pd.read_html(str(wiki))[0]\ndf.head()"

In [21]:
"""df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.index = range(len(df))
df.head()"""

"df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)\ndf.index = range(len(df))\ndf.head()"

In [17]:
'''with open('Crime_Rates.geojson') as f:
    data = geojson.load(f)
df2=pd.json_normalize(data["features"])
df2=df2.drop(columns=["type", "geometry.type", "geometry.coordinates", "properties.OBJECTID", "properties.Shape__Area", "properties.Shape__Length"])
df2.head()'''

'with open(\'Crime_Rates.geojson\') as f:\n    data = geojson.load(f)\ndf2=pd.json_normalize(data["features"])\ndf2=df2.drop(columns=["type", "geometry.type", "geometry.coordinates", "properties.OBJECTID", "properties.Shape__Area", "properties.Shape__Length"])\ndf2.head()'